# Compute 500 days averaged barotropic kinetic energy fraction for unparameterized runs

Precompute BT KE ratios for unparameterized runs (optional). Useful for faster execution of notebook `Figures7_and_C3.ipynb`.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from dask.diagnostics import ProgressBar
import warnings
warnings.filterwarnings("ignore")

In [2]:
from yamlparser import YAMLParser, read_unparameterized_runs

In [3]:
yp = YAMLParser()

In [4]:
# read unparameterized runs
exps_unparam = yp.read('/glade/u/home/noraloose/GL90paper/config_unparam.yaml')
exps_unparam = read_unparameterized_runs(exps_unparam)

## Compute barotropic vs baroclinic contributions

barotropic velocities:
$$
    u_{bt} = \frac{\sum_n h_n u_n}{\sum_n h_n}, \qquad 
    v_{bt} = \frac{\sum_n h_n v_n}{\sum_n h_n}
$$

In [5]:
for exps in [exps_unparam]:
    for exp, v in exps.items():
        ds = v['ds']
        grid = v['grid']
        st = v['st']
        h_u = grid.interp(ds['h'], 'X')
        h_v = grid.interp(ds['h'], 'Y', boundary='fill')

        ds['u_bt'] = (ds['uh'] / st.dyCu).sum(dim='zl')/ h_u.sum(dim='zl')
        ds['u_bt'] = ds['u_bt'].chunk({'yh': len(st.yh), 'xq': len(st.xq)})

        ds['v_bt'] = (ds['vh'] / st.dxCv).sum(dim='zl') / h_v.sum(dim='zl')
        ds['v_bt'] = ds['v_bt'].chunk({'yq': len(st.yq), 'xh': len(st.xh)})

barotropic KE:
$$
    KE_{bt} = \frac{1}{2} \left(\sum_n h_n\right) (u_{bt}^2 + v_{bt}^2)
$$

In [6]:
for exp, v in exps_unparam.items():
        ds_new = xr.Dataset()
        ds = v['ds']
        grid = v['grid']

        hKE = ds['h'] * ds['KE']
        
        ds_new['KE_bt'] = 0.5 * ds['h'].sum(dim='zl') * (
            grid.interp(ds['u_bt']**2, 'X')
            + grid.interp(ds['v_bt']**2, 'Y', boundary='fill')
        )

        ds_new['baro_fraction'] = ds_new['KE_bt'] / hKE.sum(dim='zl')
        
        # and some other frequently used diagnostics
        
        # u,v
        ds_new['u'] = ds['u']
        ds_new['v'] = ds['v']
                
        # uh, vh
        ds_new['uh'] = ds['uh']
        ds_new['vh'] = ds['vh']   
        
        # e
        ds_new['e'] = ds['e']
        
        v['ds_new']  = ds_new

In [12]:
for exp, v in exps_unparam.items():
    if v['degree'] == degree:
        ds_new = v['ds_new']
        dst_new = ds_new.mean(dim='time', keep_attrs=True)
        filename = '/glade/work/noraloose/nw2_%gdeg_N15_baseline_hmix20/time_averaged_diags_500days' % degree
        print(filename)
        dst_new.to_zarr(filename)

/glade/work/noraloose/nw2_0.25deg_N15_baseline_hmix20/time_averaged_diags_500days
